The first thing I need to ensure is that I can encode the state of the $ 2\times 2$ lattice.  I will label from the top left-hand corner; therefore, $|0100\rangle$ is the state with $|0\rangle$ in the vertex $(0,0)$, the top left-hand corner; $|1\rangle$ in the vertex $(0,1)$, the top right-hand corer; and, $|0\rangle$ in $(1,0)$ and $(1,1)$, the bottom left and right corners, respectively.

Key to this encoding is that $|0000\rangle$ is a tensor product of the individual states.  How can I write this state efficiently into code such that I can manipulate the state?

Luckily, $\operatorname{numpy}$ has some functions that should be useful for this task, so let's import it. 


In [2]:
# Initialisation steps
import numpy as np
from numpy import linalg as LA

Now, for an individual qubit, we assume that its state is described by a vector living in a 2-dimensional complex vector space, $V\cong \mathbb{C}^2$, where $|0\rangle = (1, 0)^T$ and $|1\rangle = (0, 1)^T$.  Knowing this information, what would we expect the state $|00\rangle$ to look like?  The theoretical answer is $|00\rangle = (1, 0, 0, 0)$.  Is that what $\operatorname{numpy}$ gives us?


In [3]:
zero_vec = (1, 0)
np.tensordot(zero_vec, zero_vec, axes=0)

array([[1, 0],
       [0, 0]])

No!

Given the complexity of this problem, let's just use an existing framework for these purposes, Qiskit.  The following code looks to create a 3 qubit circuit; recall that all qubits are initialised in the state $|0\rangle$ as standard in Qiskit.  We then apply three Hadamard gates and take note of the state.

In [92]:
import qiskit
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector

In [5]:
qc = QuantumCircuit(3)
# Apply H-gate to each qubit:
for qubit in range(3):
    qc.h(qubit)
# See the circuit:
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     └───┘

In [6]:
# Let's see the result
svsim = Aer.get_backend('qasm_simulator')
qc.save_statevector()
qobj = assemble(qc)
final_state = svsim.run(qobj).result().get_statevector()

# display this nicely using Latex.
from qiskit.visualization import array_to_latex
array_to_latex(final_state, prefix="\\text{Statevector} = ")

/Users/rossgrassie/opt/anaconda3/envs/qiskit_env/lib/python3.7/site-packages/qiskit/utils/deprecation.py:62: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  return func(*args, **kwargs)


<IPython.core.display.Latex object>

With this setup, how would we initialise our $|0000\rangle$ state for the lattice?  Try the following code. 

In [7]:
lattice = QuantumCircuit(4)
lattice.draw()

q_0: 
     
q_1: 
     
q_2: 
     
q_3:

In [8]:
lattice.save_statevector()
circ = assemble(lattice)
input_state = svsim.run(circ).result().get_statevector()
array_to_latex(input_state)

<IPython.core.display.Latex object>

This is the answer we would have expected earlier! 

To make sure I'm completely comfortable with this formalism, let's try to implement the following states:

1. $|01\rangle$
2. $|0+\rangle$
3. $|+1\rangle$
4. $|-+\rangle$


In [9]:
# Initialises the number of qubits specified in the |0> state.
qc1 = QuantumCircuit(2)
# Applies a Pauli X to the second qubit; note, they label from 0.
qc1.x(1)
# Saves the state vector at this point in the circuit.  If I move this above the X operator, the 
# steps taken later to show the final state vector will not register that the X operation took place.
qc1.save_statevector()
# Gets the circuit in a state so that it can be sent to the simulator
c1 = assemble(qc1)
# Run the simulation on the device specified for svsim, get the result and final state vector
fs1 = svsim.run(c1).result().get_statevector()
# Print the result in a latex format
array_to_latex(fs1)

<IPython.core.display.Latex object>

In [10]:
qc1.draw()

░ 
q_0: ──────░─
     ┌───┐ ░ 
q_1: ┤ X ├─░─
     └───┘ ░

This circuit will flip $q_1$ to a $|1\rangle$.  I would have expected that this would mean we would have $|01\rangle$; however, this would correspond to (0, 1, 0, 0), which is not what we have above.  What happens if we put the X operator on $q_0$?

In [11]:
qc2 = QuantumCircuit(2)
qc2.x(0)
qc2.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────

In [12]:
qc2.save_statevector()
c2 = assemble(qc2)
fs2 = svsim.run(c2).result().get_statevector()
array_to_latex(fs2)

<IPython.core.display.Latex object>

Notice that, based on these result, Qiskit takes the tensor product starting from the bottom.  This means that our states are written $|q_1 q_0\rangle$: the top qubit in the circuit diagram is always going to be the rightmost qubit in our state vector. 

With this formalism established, let's now create the second state!

In [13]:
qc3 = QuantumCircuit(2)
qc3.h(0)
qc3.draw()

┌───┐
q_0: ┤ H ├
     └───┘
q_1: ─────

Therefore, I would expect this to give me the state $|0+\rangle$.  A quick pen-and-paper calculation says that this state would be written $\frac{1}{\sqrt{2}} (1, 1, 0, 0)$.  Let's see what Qiskit gives us.

In [14]:
qc3.save_statevector()
c3 = assemble(qc3)
fs3 = svsim.run(c3).result().get_statevector()
array_to_latex(fs3)

<IPython.core.display.Latex object>

Great! We get the expected answer. 

Let's move on to the next state. 

In [15]:
qc4 = QuantumCircuit(2)
qc4.x(0)
qc4.h(1)
qc4.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ H ├
     └───┘

I now expect the final state vector to be $\frac{1}{\sqrt{2}} (0, 1, 0, 1)$, and I get

In [16]:
qc4.save_statevector()
c4 = assemble(qc4)
fs4 = svsim.run(c4).result().get_statevector()
array_to_latex(fs4)

<IPython.core.display.Latex object>

Fantastic!

Now, let's look at the final state. 

In [17]:
qc5 = QuantumCircuit(2)
qc5.x(1)
qc5.h(1)
qc5.h(0)
qc5.draw()

┌───┐     
q_0: ┤ H ├─────
     ├───┤┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

I would expect this final state vector to be $\frac{1}{2} (1, 1, -1, -1)$; is this what we get?

In [18]:
qc5.save_statevector()
c5 = assemble(qc5)
fs5 = svsim.run(c5).result().get_statevector()
array_to_latex(fs5)

<IPython.core.display.Latex object>

Great!  I think I've definitely got the two qubit case down!

Now, let's think about the four qubit case at hand. The stabilisers apply are $X^{\otimes 4}$ and $Z^{\otimes 4}$.  Can I determine the eigenspace of these operators?  Let's try to use $\operatorname{numpy}$ for this task.

First we need to get these operators in the form of matrices. Let's try to do this!

In [19]:
qc6 = QuantumCircuit(4)
for q in range(4):
    qc6.x(q)
qc6.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ X ├
     └───┘

In [20]:
qc6.save_unitary()
c6 = assemble(qc6)
stab1 = svsim.run(c6).result().get_unitary()

In [21]:
array_to_latex(stab1)

<IPython.core.display.Latex object>

Now that we have the $X^{\otimes 4}$ matrix, can I determine the eigenvectors of this system?

In [22]:
stab1

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.

In [23]:
v, w = LA.eig(stab1)

In [24]:
array_to_latex(v)

<IPython.core.display.Latex object>

In [25]:
array_to_latex(w)

<IPython.core.display.Latex object>

This is a little frustrating because of all the dots, can I do a two qubit case?

In [26]:
qc7 = QuantumCircuit(2)
for q in range(2):
    qc7.x(q)
qc7.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘

In [27]:
qc7.save_unitary()
c7 = assemble(qc7)
stab1 = svsim.run(c7).result().get_unitary()

In [28]:
v, w = LA.eig(stab1)

In [29]:
array_to_latex(v)

<IPython.core.display.Latex object>

In [30]:
array_to_latex(w)

<IPython.core.display.Latex object>

Can I pick out the +1-eigenspace?

In [31]:
print(v)

[ 1.+0.j -1.+0.j  1.+0.j -1.+0.j]


In [40]:
np.complex128(v[0])

(0.9999999999999996+0j)

This is quite strange.  I was initially having problems determining the +1-eigenspace because I could not form a Boolean that was picking out the +1 eigenvalues.  From this line of code it is clear why: I was inputting a standard integer, but we have a $\operatorname{numpy}$ complex number which is not precisely 1.  I am assuming that this result will be down to how the calculation is implemented, and the actual result is still +1, not 0.999999... . To fix this issue, I will round the number and ensure that I am comparing $\operatorname{numpy}$ complex numbers, and not different types. 

In [43]:
np.around(v[0], decimals=0)

(1+0j)

In [44]:
np.complex128(1) == np.around(v[0], decimals=0)

True

Let's pick out the +1-eigenspace!

In [88]:
es = []
for n in range(4):
    if np.complex128(1) == np.around(v[n], decimals=0):
        es = es + [w[:,n].tolist()]
out = np.transpose(np.array(es))
array_to_latex(out)

[[(0.7071067811865474-0j), (-0+0j), (-0+0j), (0.7071067811865476+0j)]]
[[(0.7071067811865474-0j), (-0+0j), (-0+0j), (0.7071067811865476+0j)], [(-0+0j), (0.7071067811865476+0j), (0.7071067811865474-0j), (-0+0j)]]


<IPython.core.display.Latex object>

Great! Here we have the two state vectors for the +1-eigenspace.  Can we determine the state written as $|\psi\rangle$?  Does that matter if Qiskit will always use the state vector representation?  Perhaps it doesn't matter.  Let's keep it as is for now. 

Now, we want to do the same for the $Z^{\otimes 2}$ stabiliser.  What is its +1-eigenspace?

In [112]:
qc8 = QuantumCircuit(2)
for q in range(2):
    qc8.z(q)
    
qc8.save_unitary()
c8 = assemble(qc8)
stab2 = svsim.run(c8).result().get_unitary()
v, w = LA.eig(stab2)

array_to_latex(v)

<IPython.core.display.Latex object>

In [111]:
array_to_latex(w)

<IPython.core.display.Latex object>

In [115]:
es2 = []
for n in range(4):
    print(v[n])
    if np.complex128(1) == np.around(v[n], decimals=0):
        es2 = es2 + [w[:,n].tolist()]
out2 = np.transpose(np.array(es2))
array_to_latex(out2)

(1+0j)
(-1+0j)
(-1+0j)
(1-0j)


<IPython.core.display.Latex object>

We now want the common +1-eigenspace of the two stabilsiers, so we need to determine if the states are the same up to global phase.  This will involve loading both sets of eigenvectors into $\operatorname{StateVector}$ objects and using the $\operatorname{equiv}$ function.

In [116]:
print(out[:,0])
print(out[:,1])

[ 0.70710678-0.j -0.        +0.j -0.        +0.j  0.70710678+0.j]
[-0.        +0.j  0.70710678+0.j  0.70710678-0.j -0.        +0.j]


Perhaps the easiest way to do this will be to put all the eigenvectors into a matrix and perform Gaussian elimination.  If there is a common eigenspace, then the rank of the matrix will be less than or equal to the size of the individual eigenspaces. 

In [123]:
from scipy.linalg import lu

matrix = np.array([ out[:,0], out[:,1], out2[:,0], out2[:,1]])
print(matrix)

[[ 0.70710678-0.j -0.        +0.j -0.        +0.j  0.70710678+0.j]
 [-0.        +0.j  0.70710678+0.j  0.70710678-0.j -0.        +0.j]
 [ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]


In [124]:
pl, u = lu(matrix, permute_l=True)

In [125]:
print(u)

[[ 1.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j -0.        +0.j  0.70710678+0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  1.        +0.j]]


Therefore, it doesn't look like there is a common eigenspace here!  We need to try a different example!

Let's try the four qubit lattice case.  We know that we should end up with a rank 2 matrix in the end!

In [129]:
bacon1 = QuantumCircuit(4)
for q in range(4):
    bacon1.x(q)
bacon1.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     ├───┤
q_3: ┤ X ├
     └───┘

In [130]:
bacon1.save_unitary()
bc1 = assemble(bacon1)
stab1 = svsim.run(bc1).result().get_unitary()
v, w = LA.eig(stab1)

es = []
for n in range(16):
    if np.complex128(1) == np.around(v[n], decimals=0):
        es = es + [w[:,n].tolist()]
out = np.transpose(np.array(es))
array_to_latex(out)

<IPython.core.display.Latex object>

In [132]:
bacon2 = QuantumCircuit(4)
for q in range(4):
    bacon2.z(q)
    
bacon2.save_unitary()
bc2 = assemble(bacon2)
stab2 = svsim.run(bc2).result().get_unitary()
v, w = LA.eig(stab2)

es2 = []
for n in range(16):
    if np.complex128(1) == np.around(v[n], decimals=0):
        es2 = es2 + [w[:,n].tolist()]
out2 = np.transpose(np.array(es2))
array_to_latex(out2)

<IPython.core.display.Latex object>

In [133]:
matrix = np.concatenate((np.transpose(out), np.transpose(out2)), axis=0)

In [135]:
array_to_latex(matrix)

<IPython.core.display.Latex object>

In [136]:
pl, u = lu(matrix, permute_l=True)
print(u)

[[1.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.70710678+0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.70710678+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.70710678+0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 1.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.

In [144]:
from numpy.linalg import matrix_rank
print(matrix.shape)
matrix_rank(u)

(16, 16)


12

In [176]:
from scipy.linalg import null_space
print(null_space(u).shape)
array_to_latex(null_space(u))

(16, 4)


<IPython.core.display.Latex object>

Okay, so we have our four-dimensional codespace embedded in the total Hilbert space for the four-qubit lattice. However, this space is really $\mathcal{H}_{s^X, s^Z = +1} = \mathcal{H}^{\mathcal{L}}_{s^X, s^Z = +1} \otimes \mathcal{H}^{\mathcal{T}}_{s^X, s^Z = +1}$.  How can we detach the single logical qubit encoded in $\mathcal{H}^{\mathcal{L}}_{s^X, s^Z = +1}$ from the gauge qubits in $\mathcal{H}^{\mathcal{T}}_{s^X, s^Z = +1}$?

Let's persist for now, maybe this will become clear when we see the action of the logical operators $\bar{X}$ and $\bar{Z}$.  Let's first get the logical circuits.

In [148]:
lx_circ = QuantumCircuit(4)
lx_circ.x(3)
lx_circ.x(1)
lx_circ.draw()

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘

In [149]:
lx_circ.save_unitary()
lx = assemble(lx_circ)
logx = svsim.run(lx).result().get_unitary()

In [150]:
lz_circ = QuantumCircuit(4)
lz_circ.z(3)
lz_circ.z(2)
lz_circ.draw()

q_0: ─────
          
q_1: ─────
     ┌───┐
q_2: ┤ Z ├
     ├───┤
q_3: ┤ Z ├
     └───┘

In [152]:
array_to_latex(null_space(u)[:,0])

<IPython.core.display.Latex object>

In [167]:
e1 = qiskit.quantum_info.Statevector(null_space(u)[:,0])
e2 = qiskit.quantum_info.Statevector(null_space(u)[:,1])
e3 = qiskit.quantum_info.Statevector(null_space(u)[:,2])
e4 = qiskit.quantum_info.Statevector(null_space(u)[:,3])

In [163]:
array_to_latex(e1.evolve(logx))

<IPython.core.display.Latex object>

In [164]:
lz_circ.save_unitary()
lz = assemble(lz_circ)
logz = svsim.run(lz).result().get_unitary()

In [165]:
array_to_latex(e1.evolve(logz))

<IPython.core.display.Latex object>

In [168]:
array_to_latex(e2.evolve(logx))

<IPython.core.display.Latex object>

In [169]:
array_to_latex(e2.evolve(logz))

<IPython.core.display.Latex object>

In [170]:
array_to_latex(e3.evolve(logx))

<IPython.core.display.Latex object>

In [172]:
array_to_latex(e3.evolve(logz))

<IPython.core.display.Latex object>

In [173]:
array_to_latex(e4.evolve(logx))

<IPython.core.display.Latex object>

In [174]:
array_to_latex(e4.evolve(logz))

<IPython.core.display.Latex object>